In [1]:
import pandas as pd
import openpyxl
import os
import paramiko
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

f:\miniconda\envs\data_analysis\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


# 取得良好标记的图片的uuid

In [5]:
wb = openpyxl.load_workbook('G:\gitcode\patho_AI\\blca.xlsx')
ws = wb.active

f:\miniconda\envs\data_analysis\lib\site-packages\openpyxl\reader\excel.py:228: UserWarning: Data Validation extension is not supported and will be removed
  ws_parser.bind_all()


In [ ]:
#获取所有标记过的数据：黄色数据的index，放在interested_cols_index这个列表里
colA = ws['A']
color_ls = []
for i in colA:
    # print(i,type(i))
    color_ls.append(i.fill.fgColor.rgb)


interested_cols_index = []
for index,color in enumerate(color_ls):
    if color == 'FFFFFF00':
        interested_cols_index.append(index)
interested_cols_index

In [ ]:
#获取所有标记为清晰、整块的数据的index
cols_quality = ws['D':'E']
# len(cols_quality)
quality_cols_index = []
for i,a in enumerate(cols_quality[0]):
    if a.value == '清晰':
        if cols_quality[1][i].value == '整块':
            quality_cols_index.append(i-1)  #这个index交给pandas处理，因为首行作为标题行不算index，所以实际index要-1
        else:
            print(f'index{i} is not a bulk')
    else:
        print(f'index{i} is not well demarcated')
quality_cols_index

In [9]:
#用pandas根据index取得uuid
df = pd.read_excel('G:\gitcode\patho_AI\\blca.xlsx')
dir_uuid = df.iloc[quality_cols_index,0]
dir_uuid.to_csv('G:\gitcode\patho_AI\\dir_uuid.csv')

f:\miniconda\envs\data_analysis\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


# 根据uuid提取相应wsi并切片到pretrainings

# 将良好标记的图片批量上传到服务器

In [13]:
def uploader(filename):
    #实例化sshclient
    ssh_client = paramiko.SSHClient()   
    # 自动添加策略，保存服务器的主机名和密钥信息，如果不添加，那么不再本地know_hosts文件中记录的主机将无法连接 ，此方法必须放在connect方法的前面
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())   
    # 连接SSH服务端，以用户名和密码进行认证 ，调用connect方法连接服务器
    ssh_client.connect(hostname='fe80::5c90:b71b:2a6e:d571', port=22, username='wangyh', password='WangYingHan01~')   
    # 打开一个Channel并执行命令  结果放到stdout中，如果有错误将放到stderr中
    stdin, stdout, stderr = ssh_client.exec_command('df -hT ') 
    # stdout 为正确输出，stderr为错误输出，同时是有1个变量有值   # 打印执行结果  print(stdout.read().decode('utf-8'))  
    sftp = ssh_client.open_sftp()
    local_path = filename
    remote_path = '/uro_biomarker/pretrainings/'
    sftp.put(local_path, remote_path)
    sftp.close()
    # 关闭SSHClient连接 
    ssh_client.close()

In [14]:
uploader('blca.xlsx')

TimeoutError: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。

In [18]:
hostname = '192.168.0.200'
host_port = 22
username = 'wangyh'
password = 'WangYingHan01~'

def ssh_client_con():
    """创建ssh连接，并执行shell指令"""
    # 1 创建ssh_client实例
    ssh_client = paramiko.SSHClient()
    # 自动处理第一次连接的yes或者no的问题
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy)

    # 2 连接服务器
    ssh_client.connect(
        port=host_port,
        hostname=hostname,
        username=username,
        password=password
    )

    # 3 执行shell命令
    # 构造shell指令
    shell_command = "ps aux"
    # 执行shell指令
    stdin, stdout, stderr = ssh_client.exec_command(shell_command)
    # 输出返回信息
    stdout_info = stdout.read().decode('utf8')
    print(stdout_info)

    # 输出返回的错误信息
    stderr_info = stderr.read().decode('utf8')
    print(stderr_info)

def sftp_client_con():
    # 1 创建transport通道
    tran = paramiko.Transport((hostname, host_port))
    tran.connect(username=username, password=password)
    # 2 创建sftp实例
    sftp = paramiko.SFTPClient.from_transport(tran)

    # 3 执行上传功能
    local_path = "'blca.xlsx'"          # 本地路径
    remote_path = "/home/wangyh/uro_biomarker/pretrainings"    # 远程路径
    put_info = sftp.put(local_path, remote_path, confirm=True)
    print(put_info)
    print(f"上传{local_path}完成")
    
    ''' 
    # 4 执行下载功能
    save_path = "7.jpg"             # 本地保存文件路径
    sftp.get(remotepath=remote_path, localpath=save_path)
    print(f'下载{save_path}完成')
    '''

    # 5 关闭通道
    tran.close()

In [19]:
ssh_client_con()
sftp_client_con()

TimeoutError: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。

In [20]:
hostname = '192.168.0.200'
host_port = 22
username = 'wangyh'
password = 'WangYingHan01~'
# 1 创建ssh_client实例
ssh_client = paramiko.SSHClient()
# 自动处理第一次连接的yes或者no的问题
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy)

# 2 连接服务器
ssh_client.connect(
    port=host_port,
    hostname=hostname,
    username=username,
    password=password
)

TimeoutError: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。